# Getting Started

## Starting sc3nb

In [ ]:
import sc3nb as scn

To startup sc3nb (sclang, scsynth, and a python OSC server) use `startup` which will return a `SC` instance which is the central SuperCollider Interface

In [ ]:
sc = scn.startup()  # see Configuration of sc3nb startup below for more details

You can produce a test sound with `blip`, which should relax any tensions whether the server is up and running. This sound should be played by the default server start.

In [ ]:
sc.server.blip()

`sc` provides you the interfaces for

* `scsynth` via `sc.server`
* `sclang` via `sc.lang`

**Configuration of sc3nb startup**:

- Specifying the excutable location
  - We try to find the sclang and scsynth executables in the `$PATH` environment variable and also in the default installation directories
      - On macOS they reside in `/Applications/SuperCollider.app/Contents` in the folders MacOS and Resources. <br>
    To add these paths to your `$PATH`, simply add to your ~/.profile, e.g. (please adapt to your installation): <br>
    `PATH=$PATH:/Applications/SuperCollider.app/Contents/MacOS:/Applications/SuperCollider.app/Contents/Resources`  
      - On Windows they reside in your Installation folder f.e `C:\Program Files\SuperCollider-3.x.x`

  - If the executables are not found, you can specify them with `sclang_path` / `scsynth_path` e.g. `sclang_path="/path/to/sclang-containing-dir/sclang"`
- You could also only use **scsynth** only and don't start **sclang** with `start_sclang=False`
- See `help(scn.startup)` for all options

In [ ]:
# help(scn.startup)

## Basic examples

### Use Python implementations to control the SuperCollider audio server

Create and control SuperCollider Synths

In [ ]:
syn = scn.Synth("s2")
syn

In [ ]:
syn.freq = 800
syn

In [ ]:
syn.free()  # or use the ctrl-. (cmd-.) shortcut in Jupyter notebooks
syn

Send SynthDefs with python code injection

In [ ]:
synth_dur = 0.5  # you can use python variables in SynthDefs by prepending ^
synth_def = scn.SynthDef('random', """{ |out|
    var line;
    line = Line.kr(1, 0, ^synth_dur, doneAction: Done.freeSelf);
    Out.ar(out, SinOsc.ar(Rand(400, 800), 0, 0.2) * line);
}""")
synth_def.add()
synth_def  # Note that the representation has synth_dur already injected

In [ ]:
scn.Synth("random")

Load a file as Buffer and play it

In [ ]:
buf = scn.Buffer().read("./media/blip.wav")
buf

In [ ]:
buf.play()

In [ ]:
buffer_data = buf.to_array()
print(buffer_data.shape)
buffer_data

More examples about the above used SuperCollider objects can be found in the respective User Guide sections 
* [Nodes (Synths and Groups)](./supercollider-objects/node-examples.ipynb)
* [SynthDef](./supercollider-objects/synthdef-examples.ipynb)
* [Buffer](./supercollider-objects/buffer-examples.ipynb)

### Use OSC to control the SuperCollider audio server

Send OSC Messages with [SuperCollider Commands](https://doc.sccode.org/Reference/Server-Command-Reference.html) and receive replies

In [ ]:
sc.server.msg("/status")  # waits for reply message and returns it

We offer also high level versions for the commands. 

In [ ]:
sc.server.status()

Create OSC Bundles using SuperCollider Commands directly

In [ ]:
with sc.server.bundler() as bundler:
    bundler.add(0.2, "/s_new", ["s1", -1, 0, 0])
    bundler.add(0.5, "/s_new", ["s1", -1, 0, 0, "freq", 800, "dur", 0.1])
bundler.messages()

or create OSC Bundles with the high level Python implementations

In [ ]:
with sc.server.bundler() as bundler:
    bundler.wait(0.2)
    scn.Synth("s1")
    bundler.wait(0.3)
    scn.Synth("s1", {"freq": 800, "dur": 0.1})
bundler.messages()

[More OSC communication examples](./osc-communication-examples.ipynb)

### Use the SuperCollider Language from Python

Execute SuperCollider Language Code

In [ ]:
breakfast = "eggs"
sc.lang.cmd('^breakfast.scramble;')

or via the **%sc** IPython magic

In [ ]:
%sc ^breakfast.scramble

Get results from SuperCollider Language in python with `cmdg` or **%scg**

In [ ]:
x = 5
value = %scg (1..^x)
print(f"received {value} with type {type(value)}")

[More sclang in sc3nb examples](./osc-communication-examples.ipynb)

### Stoping the synthesis

There are multiple options to stop the playing audio synthesis

In [ ]:
%sc x = Synth.new("default")  // starting a Synth with sclang

* to stop all playing synths either use CMD-. (in Jupyter Command mode).

* It is a shortcut for the ´free_all´ method of the default server

In [ ]:
sc.server.free_all()

* or you could also use sclang

In [ ]:
%sc s.freeAll

### Combined usage

It is also possible to mix the different interaction styles.

* create a Synth with sclang and get the nodeid of the Synth via the **%scg** IPython magic

In [ ]:
nodeid = %scg x = Synth("default"); x.nodeID;

* create the corresponding Python Synth instance

In [ ]:
synth = scn.Synth("default", nodeid=nodeid, new=False)
synth

* Inspect and control the Synth from Python

In [ ]:
synth.synth_desc

In [ ]:
synth.freq

In [ ]:
synth.freq *= 2
synth.freq

In [ ]:
synth

* send a OSC Message to free the Synth

In [ ]:
sc.server.msg("/n_free", nodeid)

In [ ]:
synth

### Exiting sc3nb

To shut down the server and sclang subprocesses

In [ ]:
sc.exit()  

## Further information and advanced examples

For more details on the specific parts please refer to the corresponding sections of the User Guide.

* Usage of [SuperCollider Language (sclang) in sc3nb](./sclang-examples.ipynb)


* Usage of the SuperCollider Objects python interface
  * [Server](./supercollider-objects/server-examples.ipynb)
  * [Nodes (Synth and Group)](./supercollider-objects/node-examples.ipynb)
  * [SynthDef](./supercollider-objects/synthdef-examples.ipynb)
  * [Buffer](./supercollider-objects/buffer-examples.ipynb)
  * [Bus](./supercollider-objects/bus-examples.ipynb)
  * [Recorder](./supercollider-objects/recorder-examples.ipynb)


* Usage of [common SuperCollider helper functions](./helper-examples.ipynb)
* Usage of [OSC in sc3nb](./osc-communication-examples.ipynb)
